In [46]:
import os
from pathlib import Path

In [26]:
%pwd

'd:\\MLops'

In [34]:
os.chdir("./MLops/chest_cancer_classification_deeplearning_project_using_dvc_mlflow")

In [35]:
%pwd

'd:\\MLops\\chest_cancer_classification_deeplearning_project_using_dvc_mlflow'

In [36]:
import tensorflow as tf

In [37]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [38]:

os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/shubhamair1996/chestcancer_prediction_using_mlflow_dvc.mlflow"
# os.environ["MLFLOW_TRACKING_USERNAME"]="xyz"
# os.environ["MLFLOW_TRACKING_PASSWORD"]="abcd"

In [39]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [40]:
from chestClassifier.constants import *
from chestClassifier.utils.common import read_yaml, create_directories, save_json

In [41]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_PATH,
        params_filepath = PARAMS_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/train",
            mlflow_uri="https://dagshub.com/shubhamair1996/chestcancer_prediction_using_mlflow_dvc.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [42]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [43]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], 
                 "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [44]:
import dagshub
dagshub.init(repo_owner='shubhamair1996', 
             repo_name='chestcancer_prediction_using_mlflow_dvc', 
             mlflow=True)

[2025-03-27 20:12:21,200: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/shubhamair1996/chestcancer_prediction_using_mlflow_dvc "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "shubhamair1996/chestcancer_prediction_using_mlflow_dvc"

[2025-03-27 20:12:21,230: INFO: helpers: Initialized MLflow to track repo "shubhamair1996/chestcancer_prediction_using_mlflow_dvc"]


Repository shubhamair1996/chestcancer_prediction_using_mlflow_dvc initialized!

[2025-03-27 20:12:21,235: INFO: helpers: Repository shubhamair1996/chestcancer_prediction_using_mlflow_dvc initialized!]


In [61]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
   raise e

[2025-03-27 20:53:01,393: INFO: common: yaml file: config\config.yaml loaded successfully]


[2025-03-27 20:53:01,406: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-27 20:53:01,412: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 24s 3s/step - loss: 0.1498 - accuracy: 0.8627
[2025-03-27 20:53:25,854: INFO: common: json file saved at: scores.json]


2025/03/27 20:53:28 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-03-27 20:53:33,291: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\Dell\AppData\Local\Temp\tmpd48stz_7\model\data\model\assets
[2025-03-27 20:53:37,913: INFO: builder_impl: Assets written to: C:\Users\Dell\AppData\Local\Temp\tmpd48stz_7\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/03/27 20:54:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 4
Created version '4' of model 'VGG16Model'.


## Moving model from staging -> productions

In [48]:

from mlflow import MlflowClient
client = MlflowClient()
# client.transition_model_version_stage(
#     name="sk-learn-random-forest-reg-model", version=3, stage="Production"
# )

In [60]:
client.transition_model_version_stage(
    name="VGG16Model", version=3, stage="Staging"
)

<ModelVersion: creation_timestamp=1743087746564, current_stage='Staging', description='', last_updated_timestamp=1743088582734, name='VGG16Model', run_id='901c71f138b54374b4ae059e5aaada21', run_link='', source='mlflow-artifacts:/8c63c2e676654f66930b94238d94aac6/901c71f138b54374b4ae059e5aaada21/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>

In [51]:
latest_mv= client.get_latest_versions("VGG16Model")[0]
print(latest_mv)

<ModelVersion: creation_timestamp=1743087746564, current_stage='None', description='', last_updated_timestamp=1743087746564, name='71909_VGG16Model', run_id='901c71f138b54374b4ae059e5aaada21', run_link='', source='mlflow-artifacts:/8c63c2e676654f66930b94238d94aac6/901c71f138b54374b4ae059e5aaada21/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>


https://www.mlflow.org/docs/latest/model-registry/#deprecated-using-model-stages

In [ ]:
# https://dagshub.com/shubhamair1996/chestcancer_prediction_using_mlflow_dvc.mlflow

# import dagshub
# dagshub.init(repo_owner='shubhamair1996', repo_name='chestcancer_prediction_using_mlflow_dvc', mlflow=True)

# import mlflow
# with mlflow.start_run():
#   mlflow.log_param('parameter name', 'value')
#   mlflow.log_metric('metric name', 1)